## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-11-01-15-01 +0000,nyt,Venezuela’s Nobel Peace Prize Winner Raises St...,https://www.nytimes.com/2025/12/10/world/europ...
1,2025-12-11-01-11-13 +0000,nypost,Hungover Russians thwart Ukraine’s ‘Pearl Harb...,https://nypost.com/2025/12/10/world-news/hungo...
2,2025-12-11-01-11-00 +0000,wsj,Oil Rises Amid Rising Tensions Between U.S. an...,https://www.wsj.com/finance/commodities-future...
3,2025-12-11-01-10-45 +0000,bbc,Trump launches $1m 'gold card' immigration visas,https://www.bbc.com/news/articles/cj4q1lddj8go...
4,2025-12-11-01-01-23 +0000,nypost,Disneyland’s Star Wars and Haunted Mansion att...,https://nypost.com/2025/12/10/us-news/star-war...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2413/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,57
49,fed,26
229,new,23
341,media,12
202,house,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
291,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...,148
35,2025-12-10-23-56-02 +0000,nypost,Trump says he wants a new owner for CNN as par...,https://nypost.com/2025/12/10/media/trump-says...,127
350,2025-12-10-07-00-26 +0000,nyt,Trump Revives Attacks on Fed Members Before In...,https://www.nytimes.com/2025/12/09/us/politics...,120
337,2025-12-10-09-00-00 +0000,cbc,Is Trump’s stark new security strategy the end...,https://www.cbc.ca/news/world/trump-world-orde...,119
384,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...,112


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
291,148,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...
335,71,2025-12-10-09-34-00 +0000,wsj,Social-Media Ban Imposes Brave New World on Au...,https://www.wsj.com/world/social-media-ban-imp...
372,66,2025-12-10-03-16-48 +0000,cbc,'Venezuela will breathe again': María Corina M...,https://www.cbc.ca/news/world/nobel-peace-priz...
151,59,2025-12-10-20-09-00 +0000,wsj,The Journal answered subscribers’ questions ab...,https://www.wsj.com/economy/central-banking/li...
81,50,2025-12-10-22-35-53 +0000,bbc,Watch: Video shows US military seizing oil tan...,https://www.bbc.com/news/videos/c9vj1jnmekko?a...
384,47,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...
255,44,2025-12-10-15-11-40 +0000,nypost,Husband charged with murdering Miss Switzerlan...,https://nypost.com/2025/12/10/world-news/husba...
55,44,2025-12-10-23-21-00 +0000,wsj,More Republicans in the House and Senate are b...,https://www.wsj.com/politics/policy/some-repub...
16,43,2025-12-11-00-32-00 +0000,wsj,Fed Chair Jerome Powell said that Fed staffers...,https://www.wsj.com/economy/jobs/fed-chair-jer...
189,43,2025-12-10-18-30-39 +0000,latimes,Democratic lawmakers say Paramount bid for War...,https://www.latimes.com/entertainment-arts/bus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
